In [23]:
import fasttext
import pandas as pd 
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
import numpy as np

In [50]:
!head -3 /workspace/datasets/reviews/output.fasttext

__label__20 not that impressive i thought this film might be interesting instead it was more of a mess the film is full of contradictions that when all is said and done just dont make any sense i think the production crew themselves might have been confused as to how they wanted to present this picture anamorphic widescreen with several bonus features but id recommended skipping it
__label__40 worth the buy while i only really enjoyed the first film you cant go wrong with this twodisc set for only 999 eddie murphy gives some memorable performances here
__label__10 incredibly disappointing based off the roald dahl childrens book this twisted and bizarre tim burton remake of the 1971 classic film is extremely poor a relatively good cast and production team goes to waste here stick with the infinitely better 1971 version starring gene wilder and titled willy wonka the chocolate factory


In [2]:
!shuf /workspace/datasets/reviews/output.fasttext > /workspace/datasets/reviews/shuffled_output.fasttext

In [51]:
!head -3 /workspace/datasets/reviews/shuffled_output.fasttext

__label__50 time capsule is excellent i replaced my linksys router with the time capsule so easy to set up amazing i really like having the router and back up hard drive all in 1 to de clutter my work station very fast router i use the capsule with my imac iphone and ipad2 excellent product i highly recommend
__label__50 everything you think it is and more fun to use and with the availability of 1000s of apps it is constantly delivering entertainment
__label__50 non blizzard wow this game feels like wow with its structure by that i mean talent trees professions mounts quests raids whats not to love for me they took all the goods from other similiar mmos and incorporated it into this game i would recommand this to someone looking for a break from wow i must warn you tho once you get into this one you will be saying wow what


In [3]:
!wc -l /workspace/datasets/reviews/shuffled_output.fasttext # 221,771

221771 /workspace/datasets/reviews/shuffled_output.fasttext


In [4]:
!head -150000 /workspace/datasets/reviews/shuffled_output.fasttext > /workspace/datasets/reviews/shuffled_output.train

In [5]:
!tail -20000 /workspace/datasets/reviews/shuffled_output.fasttext > /workspace/datasets/reviews/shuffled_output.test

In [6]:
df = pd.read_csv('/workspace/datasets/reviews/shuffled_output.fasttext')
df.columns = ['blob']
df['review'] = df['blob'].map(lambda x: x.split(' ', 1)[0])
df['comment'] = df['blob'].map(lambda x: x.split(' ', 1)[1])
print('label distribution')
print(df['review'].value_counts(normalize=True))
print('label counts')
print(df['review'].value_counts(normalize=False))

label distribution
__label__50    0.607954
__label__40    0.199256
__label__10    0.088443
__label__30    0.059494
__label__20    0.044848
__label__00    0.000005
Name: review, dtype: float64
label counts
__label__50    134826
__label__40     44189
__label__10     19614
__label__30     13194
__label__20      9946
__label__00         1
Name: review, dtype: int64


In [27]:
model = fasttext.train_supervised(input="/workspace/datasets/reviews/shuffled_output.train", 
                            lr=1.0, 
                            epoch=25, 
                            wordNgrams=2, 
#                             bucket=200000, 
#                             dim=50, 
#                             loss='hs'
                            )
test_result = model.test("/workspace/datasets/reviews/shuffled_output.test")                            
print(test_result)

Read 12M words
Number of words:  129949
Number of labels: 6
Progress: 100.0% words/sec/thread:  619929 lr:  0.000000 avg.loss:  0.240394 ETA:   0h 0m 0s 85.7% words/sec/thread:  621280 lr:  0.143110 avg.loss:  0.270789 ETA:   0h 0m 4s


(20000, 0.6888, 0.6888)


In [28]:
validation = pd.read_csv('/workspace/datasets/reviews/shuffled_output.test')
validation.columns = ['blob']
validation['review'] = validation['blob'].map(lambda x: x.split(' ', 1)[0])
validation['comment'] = df['blob'].map(lambda x: x.split(' ', 1)[1])
print('label distribution')
print(validation['review'].value_counts(normalize=True))
print('label counts')
print(validation['review'].value_counts(normalize=False))
validation.head()

label distribution
__label__50    0.605530
__label__40    0.201260
__label__10    0.088354
__label__30    0.058053
__label__20    0.046802
Name: review, dtype: float64
label counts
__label__50    12110
__label__40     4025
__label__10     1767
__label__30     1161
__label__20      936
Name: review, dtype: int64


,blob,review,comment
0,__label__40 awesome for high drain devices wit...,__label__40,everything you think it is and more fun to use...
1,__label__50 so many things to take pictures of...,__label__50,non blizzard wow this game feels like wow with...
2,__label__50 great controller i got this produc...,__label__50,ample space for two cameras memory cards etc i...
3,__label__50 great product well designed this i...,__label__50,good movie annoying preview this movie is unde...
4,__label__50 loved it it does just what i wante...,__label__50,classic thps gameplay plays like all the other...


In [29]:
validation['predicted'] = validation['comment'].map(lambda x: model.predict(x, k=1)[0][0])
validation.sample(10)

,blob,review,comment,predicted
15775,__label__40 look and sound great i have bought...,__label__40,phenomenal this movie is amazingly well done t...,__label__50
14237,__label__50 the reason to buy a psp i will adm...,__label__50,good not great this phone is definitely better...,__label__40
16705,__label__50 batteries i was actually sent an e...,__label__50,beautifully sungnaturally its pavarotti it is ...,__label__50
8505,__label__50 sony cyber shot 162 megapixel thou...,__label__50,good router for the price i had bought this to...,__label__50
1021,__label__50 great game this is a great game it...,__label__50,jeremys new remote i rated the product as high...,__label__50
16311,__label__50 great for small carpets and wood f...,__label__50,future cult movie i need to stick up for this ...,__label__50
8256,__label__50 great first person actionstrategy ...,__label__50,a good cheap hdmi cable this product works wel...,__label__50
14703,__label__50 good addition to home audio this u...,__label__50,great performance for the price i bought a pai...,__label__40
8730,__label__50 awesome ive had these headphones f...,__label__50,would be nice if it lasted this stylus is not ...,__label__20
1979,__label__50 great movie it is one of my all ti...,__label__50,solid decent but one major design flaw i bough...,__label__50


In [30]:
print(classification_report(validation.review, validation.predicted))

              precision    recall  f1-score   support

 __label__10       0.08      0.08      0.08      1767
 __label__20       0.05      0.04      0.05       936
 __label__30       0.07      0.07      0.07      1161
 __label__40       0.19      0.20      0.20      4025
 __label__50       0.60      0.60      0.60     12110

    accuracy                           0.42     19999
   macro avg       0.20      0.20      0.20     19999
weighted avg       0.42      0.42      0.42     19999



In [44]:
from collections import Counter

pd.set_option('display.max_colwidth', None)

# Lots of predictions for 5.0 when the actual label is 1.0?

validation.groupby('review').agg(
    counts = ('review', 'count'), 
    classifications = ('predicted', lambda x: Counter(x).most_common(3))
)

,counts,classifications
review,,
__label__10,1767,"[(__label__50, 1103), (__label__40, 358), (__label__10, 140)]"
__label__20,936,"[(__label__50, 565), (__label__40, 202), (__label__10, 79)]"
__label__30,1161,"[(__label__50, 663), (__label__40, 259), (__label__10, 102)]"
__label__40,4025,"[(__label__50, 2461), (__label__40, 794), (__label__10, 334)]"
__label__50,12110,"[(__label__50, 7290), (__label__40, 2484), (__label__10, 1119)]"


In [46]:
# https://www.bestbuy.com/site/reviews/the-endless-summer-ii-dvd-1994/5997406?variant=A
# https://www.bestbuy.com/site/reviews/samsung-3-7-cu-ft-9-cycle-ultra-capacity-washer-white/9742451?page=4

# our data shows a label of 1.0, but the review on the website is 5 stars
(
    validation.query('review == "__label__10"')
    .query('predicted == "__label__50"')
    .sample(10, random_state=333)
    .drop('blob', axis=1)
)

,review,comment,predicted
6584,__label__10,maximum signal everywhere in a 2 floor home installed by geek squad for free with special promotion great signal everywhere and newer computers run at n speed while older one runs at g with no signal drop,__label__50
17980,__label__10,does the job a colleague recommended something like this when my maxtor external hard drive died this is the size of a smart phone has plenty of memory was easy to install and seems to be backing everything up,__label__50
7038,__label__10,great filter took me awhile to figure that it was circular and was pleasantly surprised what it really did,__label__50
18886,__label__10,love the apps and ease of use all my family has one once we got the first one it was hard to share with everyone because of the alldifferent directions are family goes in,__label__50
12278,__label__10,32gb for 2100 i have an lg rumor phone that now has 32gb of storage my 15000 ipod has 16 gb my wife and daughter have iphone 4s with 8 gb hehehehehehehehe i got it as good as it gets i can listen to dayyyysssssss of music all 5s from me value is unparalleled capacity is only rivaled from a phone that costs hundreds more so far durability is phemominal if you upgrade your phone for hundreds when you can upgrade your phone for 2100 you might have too much money heres your sign,__label__50
5427,__label__10,great game my daughter and i enjoy playing this game i like playing this game,__label__50
6800,__label__10,definitely worth the buy you cant go wrong with a cddvd purchase i listen to this cd at least once or twice a week and ive had it since it was released its that good its like alkaline trio and dashboard confessional had a love child named bayside if you like acoustic and deep lyrics sung with raw emotion you have to get this cd then tell your friends about the cd maybe play it in your car or room with them because they will want this cd definitely worth the buy,__label__50
10618,__label__10,just get it got this computer last night and all ready have world of warcraftpureturokmercenaries 2assassins creedrunsgood but not the greatiestsporeflight sim xand gears of war on it and they all run greatit takes alittle longer then expected to start programs but its proble just cause its brand new its still really fastif u have the money expand to the full 8gigs of ram and if u want it for top end games might wana get a better graphic card the wireless work great rite out the bookmy old computer couldnt even run turok but this 1 runs it on high settings just fineassassins creed could run better but its proble just casue theres not anof graphic for itbut besides that its overall a great machine got it for only 530 with out tax and 2 year protection cant go wrong with quad core phenomits a must buy if u want a good computer without going over 800and it didnt come with a monitor cable at allkeyboard is nice if your a gamer proble want agaming keyboard the mouse is really light but works good,__label__50
12102,__label__10,unbeatable in its price range after 2 bad maytag neptunes in 10 years enough was enough after hours of research i decided on this washer and equivalent dryer boy howdy do they rock cleaner clothes almost dry when they come out quiet water and electricity efficient you can spend twice the money 1264 for both at best buy awesome i cant believe you can get twice the washerdryer my sister bought the 2400 lg wd doesnt do any better a job on anything loveem,__label__50
9923,__label__10,good computer for home use im very happy with my computer the grandkids can play their games without any problems i can work while watching tv instead of sitting at my desktop in the other room the battery life is about what was listed,__label__50
